In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('benchmark_results.csv')
df.head(10)

,model,detector,split,accuracy,n_samples
0,Facenet,retinaface,1,0.600000,20
1,Facenet,retinaface,2,0.750000,20
2,Facenet,retinaface,3,0.850000,20
3,Facenet,retinaface,4,0.750000,20
4,Facenet,retinaface,5,0.750000,20
5,Facenet,centerface,1,0.500000,16
6,Facenet,centerface,2,0.923077,13
7,Facenet,centerface,3,0.823529,17
8,Facenet,centerface,4,0.833333,18
9,Facenet,centerface,5,0.705882,17


In [ ]:
grouped_detectors = (
    df.drop_duplicates(subset=['detector', 'split'])  # nos quedamos con una fila por split-detector
      .groupby('detector')
      .agg(total_samples_detected=('n_samples', 'sum'))
      .reset_index()
)

grouped_detectors

In [19]:
grouped = df.groupby(['model', 'detector']).agg(
    avg_accuracy=('accuracy', 'mean'),
    total_samples=('n_samples', 'sum')
).reset_index()
grouped.sort_values(by=['total_samples', 'avg_accuracy'], ascending=False, inplace=True)
grouped

,model,detector,avg_accuracy,total_samples
17,VGG-Face,retinaface,0.830000,100
5,Facenet,retinaface,0.740000,100
1,ArcFace,retinaface,0.630000,100
9,GhostFaceNet,retinaface,0.360000,100
13,OpenFace,retinaface,0.200000,100
18,VGG-Face,yolov8,0.818421,99
2,ArcFace,yolov8,0.778947,99
6,Facenet,yolov8,0.767368,99
10,GhostFaceNet,yolov8,0.374211,99
14,OpenFace,yolov8,0.261053,99


In [25]:
df_times = pd.read_csv('embedding_times.csv')
df_times.head(1000)

,model,detector,subject,image_path,embedding_time_sec
0,Facenet,retinaface,Colin_Powell,/kaggle/input/dataset/merged_lfw_cplfw_50/Coli...,14.292815
1,Facenet,retinaface,Colin_Powell,/kaggle/input/dataset/merged_lfw_cplfw_50/Coli...,0.317692
2,Facenet,retinaface,Colin_Powell,/kaggle/input/dataset/merged_lfw_cplfw_50/Coli...,0.319813
3,Facenet,retinaface,Colin_Powell,/kaggle/input/dataset/merged_lfw_cplfw_50/Coli...,0.292895
4,Facenet,retinaface,Colin_Powell,/kaggle/input/dataset/merged_lfw_cplfw_50/Coli...,0.311121
...,...,...,...,...,...
995,Facenet,centerface,Gordon_Brown,/kaggle/input/dataset/merged_lfw_cplfw_50/Gord...,0.328618
996,Facenet,centerface,Alejandro_Toledo,/kaggle/input/dataset/merged_lfw_cplfw_50/Alej...,0.335673
997,Facenet,centerface,Alejandro_Toledo,/kaggle/input/dataset/merged_lfw_cplfw_50/Alej...,0.330122
998,Facenet,centerface,Alejandro_Toledo,/kaggle/input/dataset/merged_lfw_cplfw_50/Alej...,0.326605


In [22]:
mask = (df_times['model'] == 'Facenet') & (df['detector'] == 'retinaface')
df_combo = df_times[mask].copy()

# Imputar: identificar el primer registro (por índice o por fila)
# Suponiendo que está en la primera fila con esa combinación:
first_idx = df_combo.index[0]
mean_time = df_combo.loc[df_combo.index != first_idx, 'embedding_time_sec'].mean()
mean_time

np.float64(0.36624183257420856)

In [26]:
df_times.reset_index(drop=True, inplace=True)

# Agrupar por combinación (model, detector)
grouped = df_times.groupby(['model', 'detector'])

# Iterar por cada grupo y reemplazar el primer valor
for (model, detector), group_df in grouped:
    first_idx = group_df.index[0]
    
    # Obtener valor original
    original_value = df_times.loc[first_idx, 'embedding_time_sec']
    
    # Calcular el promedio sin el primer valor
    mean_time = group_df.loc[group_df.index != first_idx, 'embedding_time_sec'].mean()
    
    # Reemplazar en el DataFrame original
    df_times.loc[first_idx, 'embedding_time_sec'] = mean_time
    
    print(f"({model}, {detector}) - Valor original: {original_value:.4f} -> Reemplazado con: {mean_time:.4f}")

(ArcFace, centerface) - Valor original: 0.2647 -> Reemplazado con: 0.2544
(ArcFace, retinaface) - Valor original: 3.7819 -> Reemplazado con: 0.2122
(ArcFace, yolov8) - Valor original: 0.1371 -> Reemplazado con: 0.1419
(ArcFace, yunet) - Valor original: 0.1308 -> Reemplazado con: 0.1299
(Facenet, centerface) - Valor original: 0.8405 -> Reemplazado con: 0.3434
(Facenet, retinaface) - Valor original: 14.2928 -> Reemplazado con: 0.3668
(Facenet, yolov8) - Valor original: 15.5742 -> Reemplazado con: 0.2819
(Facenet, yunet) - Valor original: 0.6918 -> Reemplazado con: 0.2387
(GhostFaceNet, centerface) - Valor original: 0.3465 -> Reemplazado con: 0.3413
(GhostFaceNet, retinaface) - Valor original: 5.8070 -> Reemplazado con: 0.3152
(GhostFaceNet, yolov8) - Valor original: 0.2127 -> Reemplazado con: 0.2429
(GhostFaceNet, yunet) - Valor original: 0.2167 -> Reemplazado con: 0.2142
(OpenFace, centerface) - Valor original: 0.2556 -> Reemplazado con: 0.2796
(OpenFace, retinaface) - Valor original: 2

In [36]:
# Agrupar por model y detector y calcular estadísticas
summary = df_times.groupby(['model', 'detector'])['embedding_time_sec'].agg(
    mean_time='mean',
    min_time='min',
    max_time='max',
    count='count'
).reset_index()

# Ordenar de menor a mayor tiempo promedio
summary_sorted = summary.sort_values(by=['mean_time'], ascending=True)

summary_sorted


,model,detector,mean_time,min_time,max_time,count
18,VGG-Face,yolov8,0.060303,0.032361,0.335667,599
19,VGG-Face,yunet,0.061851,0.043128,0.126024,572
15,OpenFace,yunet,0.118286,0.095632,0.289626,572
14,OpenFace,yolov8,0.127524,0.084627,0.663968,599
17,VGG-Face,retinaface,0.127960,0.097930,0.316471,600
3,ArcFace,yunet,0.129886,0.106643,0.318850,572
2,ArcFace,yolov8,0.141901,0.095823,0.744107,599
16,VGG-Face,centerface,0.185367,0.159121,0.261844,582
13,OpenFace,retinaface,0.202727,0.159535,0.648590,600
1,ArcFace,retinaface,0.212191,0.165647,0.690447,600
